In this demo we will go over the basics of the Ray Job Submission Client in the SDK

In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [7]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "sha256~KAuGVI_ujl_uq57sVrOtMUN8VuNjbO00FG6P-vbhv2A",
    server = "https://api.demo-01-rhsys.wzhlab.top:6443",
    skip_tls= True
)
auth.login()

Insecure request warnings have been disabled


'Logged into https://api.demo-01-rhsys.wzhlab.top:6443'

Here, we want to define our cluster by specifying the resources we require for our batch workload. Below, we define our cluster object (which generates a corresponding RayCluster).

NOTE: The default images used by the CodeFlare SDK for creating a RayCluster resource depend on the installed Python version:

- For Python 3.9: 'quay.io/modh/ray:2.35.0-py39-cu121'
- For Python 3.11: 'quay.io/modh/ray:2.35.0-py311-cu121'

If you prefer to use a custom Ray image that better suits your needs, you can specify it in the image field to override the default.

In [8]:
# Create and configure our cluster object
# The SDK will try to find the name of your default local queue based on the annotation "kueue.x-k8s.io/default-queue": "true" unless you specify the local queue manually below
cluster = Cluster(ClusterConfiguration(
    name='jobtest',
    head_cpu_requests=1,
    head_cpu_limits=1,
    head_memory_requests=4,
    head_memory_limits=4,
    head_extended_resource_requests={'nvidia.com/gpu':0}, # For GPU enabled workloads set the head_extended_resource_requests and worker_extended_resource_requests
    worker_extended_resource_requests={'nvidia.com/gpu':0},
    num_workers=2,
    worker_cpu_requests='250m',
    worker_cpu_limits=1,
    worker_memory_requests=4,
    worker_memory_limits=4,
    image="quay.io/wangzheng422/qimgs:llama-factory-ray-20241226-v01", # Optional Field 
    write_to_file=False, # When enabled Ray Cluster yaml files are written to /HOME/.codeflare/resources 
    # local_queue="local-queue-name" # Specify the local queue manually
))

Yaml resources loaded for jobtest


Output()

In [9]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

Ray Cluster: 'jobtest' has successfully been created
Waiting for requested resources to be set up...
Requested cluster is up and running!
Dashboard is ready!


In [ ]:
cluster.status()

In [ ]:
cluster.details()

### Ray Job Submission

* Initialise the Cluster Job Client 
* Provide an entrypoint command directed to your job script
* Set up your runtime environment

In [11]:
# Initialize the Job Submission Client
"""
The SDK will automatically gather the dashboard address and authenticate using the Ray Job Submission Client
"""
client = cluster.job_client

In [12]:
# Submit an example mnist job using the Job Submission Client
submission_id = client.submit_job(
    entrypoint="echo 'wzh test'"
)
print(submission_id)

raysubmit_JMYu9X1hmpHj8A9p


In [13]:
# Get the job's logs
client.get_job_logs(submission_id)

'2024-12-26 13:36:32,341\tINFO job_manager.py:527 -- Runtime env is setting up.\nwzh test\n'

In [14]:
# Get the job's status
client.get_job_status(submission_id)

<JobStatus.SUCCEEDED: 'SUCCEEDED'>

In [15]:
# Get job related info
client.get_job_info(submission_id)

JobDetails(type=<JobType.SUBMISSION: 'SUBMISSION'>, job_id=None, submission_id='raysubmit_JMYu9X1hmpHj8A9p', driver_info=None, status=<JobStatus.SUCCEEDED: 'SUCCEEDED'>, entrypoint="echo 'wzh test'", message='Job finished successfully.', error_type=None, start_time=1735220192336, end_time=1735220194035, metadata={}, runtime_env={}, driver_agent_http_address='http://10.132.0.115:52365', driver_node_id='cf7dc2e901e9fc2494f6c6367a815c51ae194394ef52dbc37543d4b0', driver_exit_code=0)

In [16]:
# List all existing jobs
client.list_jobs()

[JobDetails(type=<JobType.SUBMISSION: 'SUBMISSION'>, job_id=None, submission_id='raysubmit_JMYu9X1hmpHj8A9p', driver_info=None, status=<JobStatus.SUCCEEDED: 'SUCCEEDED'>, entrypoint="echo 'wzh test'", message='Job finished successfully.', error_type=None, start_time=1735220192336, end_time=1735220194035, metadata={}, runtime_env={}, driver_agent_http_address='http://10.132.0.115:52365', driver_node_id='cf7dc2e901e9fc2494f6c6367a815c51ae194394ef52dbc37543d4b0', driver_exit_code=0)]

In [17]:
# Iterate through the logs of a job 
async for lines in client.tail_job_logs(submission_id):
    print(lines, end="") 

2024-12-26 13:36:32,341	INFO job_manager.py:527 -- Runtime env is setting up.
wzh test


In [ ]:
# Submit an example mnist job using the Job Submission Client
submission_id = client.submit_job(
    entrypoint="python get_pod_ip.py"
)
print(submission_id)

In [ ]:
# Delete a job
# Can run client.stop_job(submission_id) first if job is still running
client.delete_job(submission_id)

In [ ]:
cluster.down()

In [ ]:
auth.logout()